In [1]:
#import
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image
import pandas as pd

import spatial_transforms as ST
from models.ResNet import AP3DResNet50

In [2]:
## 4 Frames 
img1 = Image.open("000003.jpg")
img2 = Image.open("000004.jpg")
img3 = Image.open("000005.jpg")
img4 = Image.open("000006.jpg")

In [3]:
open("gt.txt", "r")
Koordinaten = open("gt.txt", "r")

data = pd.read_csv('gt.txt', header = None)
data.columns = ['Frame', 'ID', 'x', 'y', 'w', 'h', 'Dies', 'Das', 'Visability']

data.drop(data[data.ID != 2].index, inplace=True)
data = data.loc[data['Frame'].isin(['3','4','5','6'])]
data.drop(['Frame', 'ID', 'Dies', 'Das', 'Visability'], axis=1, inplace=True)
print(data)

x1 = data.at[1052, 'x']
y1 = data.at[1052, 'y']
w1 = data.at[1052, 'w']
h1 = data.at[1052, 'h']

x2 = data.at[1053, 'x']
y2 = data.at[1053, 'y']
w2 = data.at[1053, 'w']
h2 = data.at[1053, 'h']

x3 = data.at[1054, 'x']
y3 = data.at[1054, 'y']
w3 = data.at[1054, 'w']
h3 = data.at[1054, 'h']

x4 = data.at[1055, 'x']
y4 = data.at[1055, 'y']
w4 = data.at[1055, 'w']
h4 = data.at[1055, 'h']

        x    y   w    h
1052  372  407  80  239
1053  372  406  81  239
1054  373  405  81  239
1055  373  404  82  238


In [4]:
## hardcode 4 bounding boxes
bbox1 = img1.crop((x1, y1, x1+w1, y1+h1))
bbox2 = img2.crop((x2, y2, x2+w2, y2+h2))
bbox3 = img3.crop((x3, y3, x3+w3, y3+h3))
bbox4 = img4.crop((x4, y4, x4+w4, y4+h4))

bbox1.save('bbox1.png')
bbox2.save('bbox2.png')
bbox3.save('bbox3.png')
bbox4.save('bbox4.png')

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 128), interpolation=3),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

bbox1_trans = transform (bbox1)
bbox2_trans = transform (bbox2)
bbox3_trans = transform (bbox3)
bbox4_trans = transform (bbox4)

print(bbox1_trans.size())
print(bbox2_trans.size())
print(bbox3_trans.size())
print(bbox4_trans.size())

torch.Size([3, 256, 128])
torch.Size([3, 256, 128])
torch.Size([3, 256, 128])
torch.Size([3, 256, 128])


In [6]:
###bboxtensor = torch.stack([bbox1_trans, bbox2_trans, bbox3_trans, bbox4_trans])
bboxtensor = torch.stack([bbox1_trans, bbox2_trans, bbox3_trans])
bboxtensor.size()
bboxtensor_5dim = bboxtensor.unsqueeze(0)

bboxtensor_5dim.size()

torch.Size([1, 3, 3, 256, 128])

In [7]:
CNN3D = AP3DResNet50(625)
CNN3D.load_state_dict(torch.load('/home/ga27qef/thesis/resnet3d_mars.pth.tar')['state_dict'])

<All keys matched successfully>

In [11]:
CNN3D.eval()
features = CNN3D(bboxtensor_5dim)

features.view(6144)

tensor([0.3681, 0.3768, 0.3765,  ..., 0.2465, 0.1057, 0.0927],
       grad_fn=<ViewBackward>)